In [1]:
dbUrl = "jdbc:sqlserver://114.55.148.101:1433;databaseName=jiaowu"
df = spark.read.jdbc(dbUrl,"view_stu_course_mark",properties={"user":"wangpeng","password":"wangpeng2018!*swjtudean"})
df.createOrReplaceTempView("mark")

In [4]:
data = spark.sql("select DISTINCT(m1.student_id),m1.course_name as code1,m2.course_name as code2,m1.mark as mark1,m2.mark as mark2, \
m1.speciality_code from mark m1 join mark m2 on m1.student_id=m2.student_id \
where m1.course_code='3223700' and m2.course_code='0471024'")

In [8]:
data.cache()
data.show(5)
data.count()

+----------+-----+---------+-----+-----+---------------+
|student_id|code1|    code2|mark1|mark2|speciality_code|
+----------+-----+---------+-----+-----+---------------+
|  20092195|计算机网络|计算机网络工程实验|    8|    7|           0403|
|  20092241|计算机网络|计算机网络工程实验|    4|    4|           0403|
|  20112442|计算机网络|计算机网络工程实验|    4|    7|           0402|
|  20112722|计算机网络|计算机网络工程实验|    6|   10|           0408|
|  20122348|计算机网络|计算机网络工程实验|    9|   10|           0402|
+----------+-----+---------+-----+-----+---------------+
only showing top 5 rows



2914

In [6]:
from pyspark.sql.functions import *

In [55]:
x_good = data.filter(data.mark1>7)
x_good_cnt = x_good.count()
print(x_good_cnt)

x_bad = data.filter(data.mark1<3)
x_bad_cnt = x_bad.count()
print(x_bad_cnt)

x_y_good = x_good.filter(x_good.mark2>7)
x_y_good_cnt = x_y_good.count()
print(x_y_good_cnt)

x_y_bad = x_bad.filter(x_bad.mark2<3)
x_y_bad_cnt = x_y_bad.count()
print(x_y_bad_cnt)

# 分配同样的权重,各0.5
rxy = 0.9*(x_y_good_cnt/x_good_cnt) + 0.1*(x_y_bad_cnt/x_bad_cnt)
print("课程X-->Y的概率为:"+str(rxy))

961
665
451
190
课程X-->Y的概率为:0.45094395718745356


In [56]:
x_good = data.filter(data.mark2>7)
x_good_cnt = x_good.count()
print(x_good_cnt)

x_bad = data.filter(data.mark2<3)
x_bad_cnt = x_bad.count()
print(x_bad_cnt)

x_y_good = x_good.filter(x_good.mark1>7)
x_y_good_cnt = x_y_good.count()
print(x_y_good_cnt)

x_y_bad = x_bad.filter(x_bad.mark1<3)
x_y_bad_cnt = x_y_bad.count()
print(x_y_bad_cnt)

# 分配同样的权重,各0.5
rxy = 0.9*(x_y_good_cnt/x_good_cnt) + 0.1*(x_y_bad_cnt/x_bad_cnt)
print("课程Y-->X的概率为:"+str(rxy))

773
360
451
190
课程Y-->X的概率为:0.577874802357338


In [41]:
from pyspark.sql import SQLContext

In [57]:
# 将上述过程写成一个函数

def cal_relation(data,direction=True):
    '''True:X->Y , False:Y->X'''
    if direction:
        g_con = 'mark1>7'
        b_con = 'mark1<3'
        gg_con = 'mark2>7'
        bb_con = 'mark2<3'
    else:
        g_con = 'mark2>7'
        b_con = 'mark2<3'
        gg_con = 'mark1>7'
        bb_con = 'mark1<3'
        
    x_good = data.filter(g_con)
    x_good_cnt = x_good.count()
    print(x_good_cnt)

    x_bad = data.filter(b_con)
    x_bad_cnt = x_bad.count()
    print(x_bad_cnt)

    x_y_good = x_good.filter(gg_con)
    x_y_good_cnt = x_y_good.count()
    print(x_y_good_cnt)

    x_y_bad = x_bad.filter(bb_con)
    x_y_bad_cnt = x_y_bad.count()
    print(x_y_bad_cnt)

    # 分配权重,根据占比计算
    w_1 = x_y_good_cnt / (x_y_good_cnt + x_y_bad_cnt)
    w_2 = 1 - w_1
    try:
        rxy = w_1*(x_y_good_cnt/x_good_cnt) + w_2*(x_y_bad_cnt/x_bad_cnt)
    except ZeroDivisionError:
        print('出现除0异常')
        return 0
    return rxy

def cal_course_relation(code1,code2):
    sqlContext = SQLContext(sc)
    sqlContext.clearCache()
    data1 = spark.sql("select DISTINCT(m1.student_id),m1.course_name as code1,m2.course_name as code2,m1.mark as mark1,m2.mark as mark2, \
    m1.speciality_code from mark m1 join mark m2 on m1.student_id=m2.student_id \
    where m1.course_code='%s'  and m2.course_code='%s'" % (code1,code2))
    data1.show(5)
    x = data1.select('code1').first()['code1']
    y = data1.select('code2').first()['code2']
    data1.cache()
    rxy = cal_relation(data1)
    ryx = cal_relation(data1,False)
    print("课程 [%s-->%s]的概率为: %.4f" % (x,y,rxy))
    print("课程 [%s-->%s]的概率为: %.4f" % (y,x,ryx))

In [58]:
cal_course_relation('3223700','0471024')

+----------+-----+---------+-----+-----+---------------+
|student_id|code1|    code2|mark1|mark2|speciality_code|
+----------+-----+---------+-----+-----+---------------+
|  20092195|计算机网络|计算机网络工程实验|    8|    7|           0403|
|  20092241|计算机网络|计算机网络工程实验|    4|    4|           0403|
|  20112442|计算机网络|计算机网络工程实验|    4|    7|           0402|
|  20112722|计算机网络|计算机网络工程实验|    6|   10|           0408|
|  20122348|计算机网络|计算机网络工程实验|    9|   10|           0402|
+----------+-----+---------+-----+-----+---------------+
only showing top 5 rows

961
665
451
190
773
360
451
190
课程 [计算机网络-->计算机网络工程实验]的概率为: 0.4149
课程 [计算机网络工程实验-->计算机网络]的概率为: 0.5669


In [59]:
cal_course_relation('0471108','3273524')

+----------+-----+-------+-----+-----+---------------+
|student_id|code1|  code2|mark1|mark2|speciality_code|
+----------+-----+-------+-----+-----+---------------+
|2015111822| 操作系统|计算机组成原理|    5|    5|           0402|
|2015111940| 操作系统|计算机组成原理|    3|    7|           0402|
|2015111897| 操作系统|计算机组成原理|    4|    9|           0402|
|2015110992| 操作系统|计算机组成原理|    8|    9|           0402|
|2015111816| 操作系统|计算机组成原理|    7|    9|           0402|
+----------+-----+-------+-----+-----+---------------+
only showing top 5 rows

15
42
13
22
41
28
13
22
课程 [操作系统-->计算机组成原理]的概率为: 0.6512
课程 [计算机组成原理-->操作系统]的概率为: 0.6116


In [60]:
cal_course_relation('0473003','0473004')

+----------+----------+------------+-----+-----+---------------+
|student_id|     code1|       code2|mark1|mark2|speciality_code|
+----------+----------+------------+-----+-----+---------------+
|  20112442|嵌入式系统设计与应用|嵌入式系统设计与应用实验|    1|    1|           0402|
|  20122348|嵌入式系统设计与应用|嵌入式系统设计与应用实验|   10|   10|           0402|
|  20122380|嵌入式系统设计与应用|嵌入式系统设计与应用实验|    1|    0|           0402|
|  20122380|嵌入式系统设计与应用|嵌入式系统设计与应用实验|    1|    6|           0402|
|  20131986|嵌入式系统设计与应用|嵌入式系统设计与应用实验|    9|    6|           0402|
+----------+----------+------------+-----+-----+---------------+
only showing top 5 rows

193
357
98
177
170
288
98
177
课程 [嵌入式系统设计与应用-->嵌入式系统设计与应用实验]的概率为: 0.5001
课程 [嵌入式系统设计与应用实验-->嵌入式系统设计与应用]的概率为: 0.6010


In [61]:
cal_course_relation('0473097','3273523')

+----------+-------+-----+-----+-----+---------------+
|student_id|  code1|code2|mark1|mark2|speciality_code|
+----------+-------+-----+-----+-----+---------------+
|2015111822|算法分析与设计| 数据结构|    4|    6|           0402|
|2015111940|算法分析与设计| 数据结构|    3|    5|           0402|
|2015111897|算法分析与设计| 数据结构|    7|    7|           0402|
|2015110992|算法分析与设计| 数据结构|   10|    8|           0402|
|2015111816|算法分析与设计| 数据结构|    7|    7|           0402|
+----------+-------+-----+-----+-----+---------------+
only showing top 5 rows

32
62
18
35
28
43
18
35
课程 [算法分析与设计-->数据结构]的概率为: 0.5638
课程 [数据结构-->算法分析与设计]的概率为: 0.7558
